### 모델 라이브러리

In [58]:
# 모델 라이브러리
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier, AdaBoostClassifier

# 보팅
from sklearn.ensemble import VotingClassifier

# 스테킹
from sklearn.ensemble import StackingClassifier

In [65]:
import optuna
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

def objectiveCatBoost(trial, x_tr, y_tr, x_val, y_val):
    param = {
        'iterations': trial.suggest_int('iterations', 100, 800)
        , 'depth': trial.suggest_int('depth', 2, 16)
        , 'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2)
        , 'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 0.01, 10.0)
        , 'border_count': trial.suggest_int('border_count', 10, 100)

        , 'random_seed': 0
        , 'verbose': False
    }
    
    model = CatBoostClassifier(**param)
    model.fit(x_tr, y_tr)
    pred = model.predict(x_val)
    score = f1_score(y_val, pred, average="binary")
    
    return score

# 데이터셋 분할
x_train, x_val, y_train, y_val = train_test_split(
    df_train_encoded.drop("is_converted", axis=1),
    df_train_encoded["is_converted"].astype(int),
    test_size=0.2,
    shuffle=True,
    random_state=400,
)

# 하이퍼 파라미터 튜닝
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=0))
study.optimize(lambda trial: objectiveCatBoost(trial, x_train, y_train, x_val, y_val), n_trials=300)

print('Best trial: score {}, \nparams {}'.format(study.best_trial.value, study.best_trial.params))


[I 2024-02-22 17:13:16,027] A new study created in memory with name: no-name-fa32ef68-b720-4e50-b394-3f5c0afd3e96


[I 2024-02-22 17:14:02,619] Trial 0 finished with value: 0.8334274421230944 and parameters: {'iterations': 484, 'depth': 12, 'learning_rate': 0.12452504145361233, 'l2_leaf_reg': 0.4311710058685491, 'border_count': 48}. Best is trial 0 with value: 0.8334274421230944.
[I 2024-02-22 17:14:11,257] Trial 1 finished with value: 0.8336153412295544 and parameters: {'iterations': 552, 'depth': 8, 'learning_rate': 0.17943687014859516, 'l2_leaf_reg': 7.780155576901417, 'border_count': 44}. Best is trial 1 with value: 0.8336153412295544.
[I 2024-02-22 17:14:26,452] Trial 2 finished with value: 0.832116788321168 and parameters: {'iterations': 654, 'depth': 9, 'learning_rate': 0.11792846660784713, 'l2_leaf_reg': 5.981221901152554, 'border_count': 16}. Best is trial 1 with value: 0.8336153412295544.
[I 2024-02-22 17:14:27,962] Trial 3 finished with value: 0.7870425914817036 and parameters: {'iterations': 161, 'depth': 2, 'learning_rate': 0.16819777065410824, 'l2_leaf_reg': 2.1600820741402047, 'border

KeyboardInterrupt: 

### 모델 성능확인

In [66]:
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)

def get_clf_eval(y_test, y_pred=None):
    confusion = confusion_matrix(y_test, y_pred, labels=[True, False])
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, labels=[True, False])
    recall = recall_score(y_test, y_pred)
    F1 = f1_score(y_test, y_pred, labels=[True, False])
    weighted_F1 = f1_score(y_test, y_pred, average='weighted')

    metrics = pd.DataFrame({
        '정확도': [accuracy],
        '정밀도': [precision],
        '재현율': [recall],
        'F1 Score': [F1],
        'Weighted F1': [weighted_F1]
    })

    confusion_df = pd.DataFrame(confusion, index=['True', 'False'], columns=['True', 'False'])

    print("\n오차행렬:")
    display(confusion_df)
    print("평가 지표:")
    display(metrics)

### 모델 정의

Optuna를 이용하여 파라미터를 구함

In [71]:
# CatBoost
model = CatBoostClassifier(
    iterations=308
    , depth=12
    , learning_rate=0.14214840217472086
    , l2_leaf_reg= 3.4914313211595593
    , border_count = 95
    , verbose=False
    , random_state=0
)

### 모델 학습

In [72]:
model.fit(x_train, y_train)

In [73]:
pred = model.predict(x_val)
get_clf_eval(y_val, pred)


오차행렬:


,True,False
True,747,200
False,78,10835


평가 지표:


,정확도,정밀도,재현율,F1 Score,Weighted F1
0,0.97656,0.905455,0.788807,0.843115,0.975818


In [74]:
# 예측에 필요한 데이터 분리
x_test = df_test_encoded.drop(["is_converted", "id"], axis=1)

test_pred = model.predict(x_test)
sum(test_pred) # True로 예측된 개수

598

### 제출파일 생성

In [75]:
# 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
df_sub = pd.read_csv("./data/submission.csv")
df_sub["is_converted"] = test_pred

# 제출 파일 저장
df_sub.to_csv("submission_cat_optuna.csv", index=False)

.